# Extract anchors
Extract the anchors from a group of strings. Anchors are the common substrings among all the given strings.

In [1]:
import numpy as np

from ga import GA_MSA
from os import listdir
from ntpath import basename
from os.path import isfile, join
from collections import defaultdict

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

In [2]:
data_dir = join("..", "msa_data")

ref_dir = join(data_dir, "Ref11")

true_aln_dir = join(ref_dir, "true_alignment")
unaligned_dir = join(ref_dir, "unaligned")
clustalo_dir = join(ref_dir, "clustalo")
muscle_dir = join(ref_dir, "muscle")
mafft_dir = join(ref_dir, "mafft")
prank_dir = join(ref_dir, "prank")
tcoffee_dir = join(ref_dir, "tcoffee")
xml_dir = join(ref_dir, "xml")
output_dir = join(ref_dir, "anchor_ga")

def get_files(dir_name):
    return [join(dir_name, f) for f in listdir(dir_name) if isfile(join(dir_name, f))]

true_aln_files = get_files(true_aln_dir)
unaligned_files = get_files(unaligned_dir)
clustalo_files = get_files(clustalo_dir)
muscle_files = get_files(muscle_dir)
mafft_files = get_files(mafft_dir)
prank_files = get_files(prank_dir)
tcoffee_files = get_files(tcoffee_dir)
xml_files = get_files(xml_dir)
output_files = get_files(output_dir)

aligners_list = [clustalo_files, muscle_files, mafft_files, prank_files, tcoffee_files]


In [3]:
def get_file_list(file_id):
    return [aligner[file_id] for aligner in aligners_list]

def sequence_dict(file_list):
    alignments = defaultdict(list)
    for f in file_list:
        with open(f, "rU") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                alignments[record.id].append(str(record.seq))

    return alignments

def alignments_array(seq_dict):
    alignments = defaultdict(list)
    for k in seq_dict:
        for (i, aln) in enumerate(seq_dict[k]):
            alignments[i].append(aln)
    aln_array = np.array([np.array(l) for l in alignments.values()])
    return aln_array

def has_equal_sizes(aln_array):
    for l in aln_array:
        if not all([len(l[0]) == len(s) for s in l]):
            return False
    return True

def substring(seq_list, start=0, window=3):
    st = ""
    for l in seq_list:
        st += l[start:start+window]
    return st

def aln_substring(aln_array, start=0, window=3, search_area=10):
    aln_substrs = np.array([substring(seq, start=start, window=window) for seq in aln_array])
    min_index = np.argmin([len(l[0]) for l in aln_array])
    anchor_substring = aln_substrs[min_index]
    equal_to_anchor = aln_substrs == anchor_substring
    if not np.all(equal_to_anchor):
        for i in np.where(~(equal_to_anchor))[0]:
            for sa in range(1, min(search_area, len(aln_array[i][0]))):
                new_substr = substring(aln_array[i], start=(start+sa), window=window)
                if new_substr == anchor_substring:
                    aln_substrs[i] = new_substr
                    break

    return aln_substrs

def get_anchors(aln_array):
    min_size = np.min([len(l[0]) for l in aln_array])
    anchors = list()
    s = 0
    w = 3
    
    while s < min_size:
        aligned_sub = False
        aln_substr = aln_substring(aln_array, start=s, window=w)
        while np.all(aln_substr == aln_substr[0]):
            aligned_sub = True
            w += 1
            aln_substr = aln_substring(aln_array, start=s, window=w)
        if aligned_sub:
            anchors.append((s, w-1))
            s += w
        else:
            s += 1
    return anchors
        
def get_unaligned_pos(aln_array, anchors):
    sizes = [len(l[0]) for l in aln_array]
    min_size = np.min(sizes)
    min_index = np.argmin(sizes)
    aln = aln_array[min_index]
    sequences = list()
    s = 0
    for (anchor_start, anchor_window) in anchors:
        sequences.append((s, anchor_start))
        s = anchor_start + anchor_window
    sequences.append((s, min_size))
    return sequences

def get_unaligned_seq(aln_array, unaln_pos):
    sequences = defaultdict(list)
    min_index = np.argmin([len(l[0]) for l in aln_array])
    for (i, seq) in enumerate(aln_array[min_index]):
        for (s, e) in unaln_pos:
            sequences[i].append(seq[s:e])
    seq_array = [[] for i in range(len(sequences[0]))]
    for seq in sequences.values():
        for (i, n) in enumerate(seq):
            seq_array[i].append(n.replace("-", ""))
    return seq_array

def align_ga(unaln_seq):
    ga = GA_MSA(population_size=20, generations=100, min_generations=20, mutation_rate=0.05, gap_open_score=-2, gap_extend_score=-1)

    aln_seq = list()
    for seq in unaln_seq:
        score, aln = ga.run(sequences=seq)
        aln_seq.append(aln)
    return aln_seq

def concatenate(aln_array, anchors, aln_seq):
    num_seq = len(aln_array[0])
    alignments = ["" for i in range(num_seq)]
    min_index = np.argmin([len(l[0]) for l in aln_array])
    for i in range(len(anchors)):
        for j in range(num_seq):
            start, window = anchors[i]
            alignments[j] += aln_seq[i][j] + aln_array[min_index][j][start: start+window]
    for i in range(num_seq):
        alignments[i] += aln_seq[-1][i]
    return alignments

def anchor_ga(file_id):
    seq_dict = sequence_dict(get_file_list(file_id))
    aln_array = alignments_array(seq_dict)
    anchors = get_anchors(aln_array)
    unaln_pos = get_unaligned_pos(aln_array, anchors)
    unaln_seq = get_unaligned_seq(aln_array, unaln_pos)

    aln_seq = align_ga(unaln_seq)

    alignments = concatenate(aln_array, anchors, aln_seq)

    sequence_records = list()
    ids = list(seq_dict)
    for i, aln in enumerate(alignments):
        sequence_records.append(SeqRecord(Seq(aln,
                        IUPAC.protein),
                    id=ids[i], description=""))
    output_file = join(output_dir, basename(true_aln_files[file_id]))
    with open(output_file, "w") as output_handle:
        SeqIO.write(sequence_records, output_handle, "fasta")
    print("Output saved to: " + str(output_file))

    return alignments

def extract_score(score_str):
    sop = "sum_of_pairs: "
    col = "column_score: "
    sop_pos = score_str.find(sop) + len(sop)
    col_pos = score_str.find(col)
    sop_value = float(score_str[sop_pos:col_pos-2])
    col_pos += len(col)
    col_value = float(score_str[col_pos:-2])
    return (sop_value, col_value)


In [6]:
scores = dict()
for file_id in range(3):
    alignments = anchor_ga(file_id)
    result = ! bali-score -r {xml_files[0]} -t {output_files[0]}
    scores[file_id] = extract_score(result[0])

Running

Best solution:
GKGD--P-KKPRGK
-MQD-R-VKRP---
MKKLKKHPDFPKKP
-----MHIKKP---
Running

Best solution:
PDASVNFS
PRM-RNS-
PEMS-NL-
T-L-KESA
Running

Best solution:
RYEREMKTYIP---PK---G-------E-
MH-RE---K--Y-PNYKYRPRRKAKMLPK
EFE-RNLARF-REDH-PDLIQ---NA-KK
LH---MQLY-PGWSARDNYGKKKKRKREK
Output saved to: ..\msa_data\Ref11\anchor_ga\BB11001.fas
Running

Best solution:
-------------------N-L-F---V----------ALY---DF-V--AS---G-D-N--T--L-S-I-TK--G------------EKL-RV-LG----YNH---NG-E--WC----------E-A-Q-T-KNG-------Q---------GWVPS--NY-I--------------------T--PV-N---S-----
PLALLLDSSLEGEFDLVQ-RIIYEVD---DPSLPNDEGITALHNAVCAGHTEIVKFLVQFGVNVNAADSDGWTPLHCAASCNNVQVCKFLVES-GAAVFAMTYSDMQTAADKCEEMEEGYTQCSQFLYGVQEKMGIMNKGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDE-IEWWWARLNDKEGYV-PRNLLGLYP
--A-------EG-Y---QYRALY--DYKKE----REEDID-LH--L--G--DI---LT---VN------KG--------S---------LV-ALG---F----SDGQEA--RPEEI--GW------LNGYNETTG--ERG------DF-P----------G---TYV------EYI----GR---KK--ISP-------P
MI-Q-N----F-----R---------VY

In [7]:
scores

[{'sum_of_pairs: ': 0.8801169590643275, 'column_score: ': 0.8771929824561403}]

In [5]:
fid = 2
seq_dict = sequence_dict(get_file_list(fid))
aln_array = alignments_array(seq_dict)
has_equal_sizes(aln_array)
aln_strs = aln_substring(aln_array, start=14, window=19)
anchors = get_anchors(aln_array)
anchors
unalnpos = get_unaligned_pos(aln_array, anchors)
unalnpos
unaln_seq = get_unaligned_seq(aln_array, unalnpos)
unaln_seq
aln_seq = align_ga(unaln_seq)
aln_seq

Running

Best solution:
-S-ISDTVK-RAREAFNSGKTR--SLQFRI--Q-QLEALQRMI--NENLKS-ISG--ALASDLGKNE--------WTSY-----Y-EE------V-AHVL-----EELDTT-IKEL-PDWAE---D--EPV------AKT--R-Q----TQ------QDD--LYIHSEPLGV-VLVIGA-WNYP---FNLTIQPM-VGAVAAGNAVILKPSE----VSG-------HMADLLATLIPQ-YMDQNLYLVVKG-GVPETTEL---LKE----RFDHIMYTGSTAVG-KIVMAAAAKHLTP-VT------LELGGKSPCYVDKD-CDLDVACRRI---AWGKFMNSGQTCVAPD-YILC----DPSIQNQIVEK-LKKS--LKDFYGEDAKQSRDYGRIIN--DRHFQRVKGL--ID---N--QKVAHGGTWDQSSR-----Y-IAPTILVDVDPQSP---VMQEEIFGPVMPIVCVRSLEEAIQFI-N-------------QRE--KPL--A--------LYVFSNNEKVIKKMIAETSSGGVTANDVIV-HITVPTLPFGG--V-G-NSGMGA--YHGKKSFETFSHRRSCL-V-KSLLNEEAHKARYPPSPA
RAMRE-ALRRVREEFGRHYPLYIGGEWVDTKERMVSL-NPSAPSEVVGTTA---KA-GK-AEA--E---AALEA----AWKAFK---TWKDWPQE---DRSRLLLKAAALMRRRKR--E-LEA--TLVYEVG-KNWVEASAD-VAEAID-FIEYYARA-ALRY-RYPAVEVVPYP-GEDNESFYVPLGAGVVI--A-PWNFPVAIFTGMIVGPVAV--GNTVIAKPAEDAV-----VVGAKVFEIFHEAGFPPGVVNFLPGVGE-EVGAYLVEHPR--IRFI-NFTG-S--LE-VGLKIYEAAGRLA---P--GQTW----FKR--AYV---ETG-GKNA-IIVDETADFDL--AAEGVVVSA

[['-', 'M', '-', '-'],
 ['-S-ISDTVK-RAREAFNSGKTR--SLQFRI--Q-QLEALQRMI--NENLKS-ISG--ALASDLGKNE--------WTSY-----Y-EE------V-AHVL-----EELDTT-IKEL-PDWAE---D--EPV------AKT--R-Q----TQ------QDD--LYIHSEPLGV-VLVIGA-WNYP---FNLTIQPM-VGAVAAGNAVILKPSE----VSG-------HMADLLATLIPQ-YMDQNLYLVVKG-GVPETTEL---LKE----RFDHIMYTGSTAVG-KIVMAAAAKHLTP-VT------LELGGKSPCYVDKD-CDLDVACRRI---AWGKFMNSGQTCVAPD-YILC----DPSIQNQIVEK-LKKS--LKDFYGEDAKQSRDYGRIIN--DRHFQRVKGL--ID---N--QKVAHGGTWDQSSR-----Y-IAPTILVDVDPQSP---VMQEEIFGPVMPIVCVRSLEEAIQFI-N-------------QRE--KPL--A--------LYVFSNNEKVIKKMIAETSSGGVTANDVIV-HITVPTLPFGG--V-G-NSGMGA--YHGKKSFETFSHRRSCL-V-KSLLNEEAHKARYPPSPA',
  'RAMRE-ALRRVREEFGRHYPLYIGGEWVDTKERMVSL-NPSAPSEVVGTTA---KA-GK-AEA--E---AALEA----AWKAFK---TWKDWPQE---DRSRLLLKAAALMRRRKR--E-LEA--TLVYEVG-KNWVEASAD-VAEAID-FIEYYARA-ALRY-RYPAVEVVPYP-GEDNESFYVPLGAGVVI--A-PWNFPVAIFTGMIVGPVAV--GNTVIAKPAEDAV-----VVGAKVFEIFHEAGFPPGVVNFLPGVGE-EVGAYLVEHPR--IRFI-NFTG-S--LE-VGLKIYEAAGRLA---P--GQTW----FKR--AYV---ETG-GKNA-IIVDETADFDL--AA

In [12]:
output = []
result = ! bali-score -r {xml_files[0]} -t {output_files[0]}
result = result[0]
result

'AlignmentScores { sum_of_pairs: 0.8538011695906432, column_score: 0.8070175438596491 }'

In [6]:
max([0, 1, 0, 0])

1

In [119]:
result[col_pos:-2]

'0.8596491228070176'

In [7]:
first_file['1k99A']

['MKKLKKHPDFPKKPLTPYFRFFMEKRAKYAKLHPE--MSNLDLTKILSKKYKELPEKKKMKYIQDFQREKQEFERNLARFREDHPDLIQNAKK---',
 'MKKLKKHPDFPKKPLTPYFRFFMEKRAKYAKLHPEMS-NL-DLTKILSKKYKELPEKKKMKYIQDFQREKQEFERNLARFREDH---PDLIQNAKK',
 'MKKLKKHPDFPKKPLTPYFRFFMEKRAKYAKLHPEMS-NL-DLTKILSKKYKELPEKKKMKYIQDFQREKQ-------------EFERNLARFREDHPDLIQNAKK',
 'MKKLKKHPDFPKKP---LTPYFRFFMEKRAKYAKLHPEMSN--LDLTKILSKKYKELPEKKKMKYIQDFQREKQEFERNLARFREDH-P--DLIQNAKK-------------',
 'MKKLKKHPDFPK---KPLTPYFRFFMEKRAKYAKLHPEM--SNLDLTKILSKKYKELPEKKKMKYIQDFQREKQEFERNLARFREDHPDLI----------QNAKK']

In [8]:
first_file['1aab']

['---GKGDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKARYEREMKTYIPPKGE----------',
 '---GKGDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKARYEREMKTYI------PPKGE----',
 '---GKGDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKA-------------RYEREMKTY-------IPPKGE',
 '------GKGDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKARYE----REMKTYIP-------------------PKGE',
 'GK------GDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKARYER-------EMKTYIP---------PKG-E']

In [15]:
first_file['1j46A']

['------MQDRVKRPMNAFIVWSRDQRRKMALENPR--MRNSEISKQLGYQWKMLTEAEKWPFFQEAQKLQAMHREKYPNYKYRPRRKAKMLPK---',
 '------MQDRVKRPMNAFIVWSRDQRRKMALENPRMR-NS-EISKQLGYQWKMLTEAEKWPFFQEAQKLQAMHREKYPNYKYRP---RRKAKMLPK',
 'MQ------DRVKRPMNAFIVWSRDQRRKMALENPRMR-NS-EISKQLGYQWKMLTEAEKWPFFQEAQKLQAMHREKYPNYKYRP---RRKAKM-------LP---K',
 '------MQDRVKRP---MNAFIVWSRDQRRKMALENPRMRN--SEISKQLGYQWKMLTEAEKWPFFQEAQKLQA-------MHREKY-P------NYKYRPRRKAKMLPK--',
 'MQ------DRVK---RPMNAFIVWSRDQRRKMALENPRM--RNSEISKQLGYQWKMLTEAEKWPFFQEAQKLQAMHRE-------KYPNYKYRP---RRKAKMLPK']

In [19]:
alignment_dict([unaligned_files[0]])

defaultdict(list,
            {'1aab_': ['GKGDPKKPRGKMSSYAFFVQTSREEHKKKHPDASVNFSEFSKKCSERWKTMSAKEKGKFEDMAKADKARYEREMKTYIPPKGE'],
             '1j46_A': ['MQDRVKRPMNAFIVWSRDQRRKMALENPRMRNSEISKQLGYQWKMLTEAEKWPFFQEAQKLQAMHREKYPNYKYRPRRKAKMLPK'],
             '1k99_A': ['MKKLKKHPDFPKKPLTPYFRFFMEKRAKYAKLHPEMSNLDLTKILSKKYKELPEKKKMKYIQDFQREKQEFERNLARFREDHPDLIQNAKK'],
             '2lef_A': ['MHIKKPLNAFMLYMKEMRANVVAESTLKESAAINQILGRRWHALSREEQAKYYELARKERQLHMQLYPGWSARDNYGKKKKRKREK']})

[]